# 分子動力学プログラム LAMMPS を Amazon SageMaker Processing で GPU を用いて動かすサンプル

* [LAMMPS](https://www.lammps.org/) を Amazon SageMaker Processing で動かす。細かい情報は下記を参照。
    * [LAMMPSのマニュアル](https://docs.lammps.org/Manual.html)
    * [ポリエチレン分子をシミュレーション](https://winmostar.com/jp/tutorials/LAMMPS_tutorial_8%28Polymer_Elongation%29.pdf)
    * [SageMaker SDK doc](https://sagemaker.readthedocs.io/en/stable/amazon_sagemaker_processing.html)
    * [SageMaker Processing 開発者ガイド](https://docs.aws.amazon.com/sagemaker/latest/dg/processing-job.html)

このノートブックは Amazon SageMaker Notebook(≠Studio) の GPU インスタンス(g4dn.xlargeなど)で実行することを前提とする。理由は以下の通り。

1. docker コマンドを使うため(Studioを使う場合は sm-docker コマンドに修正の必要あり)
2. ビルドしたコンテナをローカル(SageMaker Notebook内)でテスト実行するのにあたり、GPU で動かす必要があるため

## コンテナイメージのビルド
まずは SageMaker Processing で LAMMPS が動かせるよう、コンテナイメージの中で LAMMPS をビルドする。  
### ビルド環境のセットアップ
ビルドは、`/var/lib/docker`を利用するが、SageMaker Notebook では該当領域は`/` にマウントされた 15GB では、ビルドに耐えられないので、別途 EBS をマウントしている`/home/ec2-user/SageMaker`以下の領域を使うように変更するスクリプトを実行する

In [ ]:
cat ./setup.sh

In [ ]:
!./setup.sh

### ビルド

In [ ]:
import boto3, sagemaker, os
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput

In [8]:
%%time

IMAGE_NAME = 'lampps'
TAG=':v1'

%cd ./container
!docker stop $(docker ps -q)
!docker rm $(docker ps -q -a)
# !docker rmi -f $(docker images -a -q)
!docker build -t {IMAGE_NAME}{TAG} .
%cd ../

/home/ec2-user/SageMaker/lampps-sagemaker/container
"docker stop" requires at least 1 argument.
See 'docker stop --help'.

Usage:  docker stop [OPTIONS] CONTAINER [CONTAINER...]

Stop one or more running containers
"docker rm" requires at least 1 argument.
See 'docker rm --help'.

Usage:  docker rm [OPTIONS] CONTAINER [CONTAINER...]

Remove one or more containers
Sending build context to Docker daemon  3.072kB
Step 1/4 : FROM nvidia/cuda:11.4.2-devel-ubuntu20.04
 ---> 859b6b91d5c1
Step 2/4 : ENV DEBIAN_FRONTEND=noninteractive
 ---> Using cache
 ---> 4a7a0c0873a8
Step 3/4 : RUN apt-get -y update &&     apt-get -y upgrade &&     apt-get -y install python3.9 python3-pip wget cmake build-essential libopenmpi-dev software-properties-common --no-install-recommends &&     rm -rf /var/lib/apt/lists/* &&     ln -s /usr/bin/python3.9 /usr/bin/python &&     python -m pip install numpy pandas &&     mkdir -p /program/lammps201029 &&     cd /program/lammps201029 &&     wget https://download.lammps.

### Image Test
ビルドしたイメージをテストする。  
このインスタンスでコンテナを動かし、mpirunコマンド(`./lmp_equiliv.sh`に内包)を実行し実際にシミュレーションを行う。  

jupyter notebook の new -> terminal から下記コマンドを実行
```
cd /home/ec2-user/SageMaker/lampps-sagemaker # clone した先のディレクトリ
docker run --gpus all -v /home/ec2-user/SageMaker/lampps-sagemaker/test/:/test -it --rm --entrypoint "bash" lampps:latest
cd /test
./lmp_equiliv.sh

tail -n1 lmp_equiliv.log # 実行時間が出ていたらOK, 概ね g4dn.xlarge で 6 分弱で完了
tail -n1 lmp2data_equiliv.log  # finish replica が出ていたらOK

exit # 終わったら exit でコンテナから抜ける(コンテナも終了する)
```

### Push

In [9]:
%%time

MY_ACCOUNT_ID = boto3.client('sts').get_caller_identity().get('Account')
# PUBLIC_ACCOUNT_ID = '763104351884'

REGION = boto3.session.Session().region_name

MY_ECR_ENDPOINT = f'{MY_ACCOUNT_ID}.dkr.ecr.{REGION}.amazonaws.com/'



MY_REPOSITORY_URI = f'{MY_ECR_ENDPOINT}{IMAGE_NAME}'
MY_IMAGE_URI = f'{MY_REPOSITORY_URI}{TAG}'

!$(aws ecr get-login --region {REGION} --registry-ids {MY_ACCOUNT_ID} --no-include-email)
 
# リポジトリの作成
!aws ecr delete-repository --repository-name {IMAGE_NAME} --force # 同名のリポジトリがあった場合削除
!aws ecr create-repository --repository-name {IMAGE_NAME}
 
# !docker push $image_uri
!docker tag {IMAGE_NAME}{TAG} {MY_IMAGE_URI}
!docker push {MY_IMAGE_URI}

print(f'コンテナイメージは {MY_IMAGE_URI} へ登録されています。')

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
{
    "repository": {
        "repositoryArn": "arn:aws:ecr:ap-northeast-1:155580384669:repository/lampps",
        "registryId": "155580384669",
        "repositoryName": "lampps",
        "repositoryUri": "155580384669.dkr.ecr.ap-northeast-1.amazonaws.com/lampps",
        "createdAt": 1634024924.0,
        "imageTagMutability": "MUTABLE"
    }
}
{
    "repository": {
        "repositoryArn": "arn:aws:ecr:ap-northeast-1:155580384669:repository/lampps",
        "registryId": "155580384669",
        "repositoryName": "lampps",
        "repositoryUri": "155580384669.dkr.ecr.ap-northeast-1.amazonaws.com/lampps",
        "createdAt": 1634138014.0,
        "imageTagMutability": "MU

37eb2256: Pushing  1.105GB/3.01GBBPushing  75.16MBPushing  1.043GB/2.072GB

ad83e92: Pushing  2.174GB/2.568GBPushing  1.444GB/2.568GBPushing  2.158GB/2.568GB

37eb2256: Pushed   3.016GB/3.01GBBlatest: digest: sha256:7a177b43595ae9a98db1d002cc5dae4586c1fe913fdd5fb09054675ab68c1a0a size: 2427
コンテナイメージは 155580384669.dkr.ecr.ap-northeast-1.amazonaws.com/lampps:latest へ登録されています。
CPU times: user 2.49 s, sys: 747 ms, total: 3.24 s
Wall time: 2min 41s


## SageMaker Processing で ポリエチレンのシミュレーション
* コンテナイメージの作成が完了したので、シミュレーションのジョブを投入する
* 以降重い処理はこのノートブックインスタンスで実行することはなく、GPUを使うこともないので、ただジョブの投入を行うだけなので、t3.mediumなどでも十分動く

### 必要なデータ(パラメータファイルなど)を S3 にアップロード

In [20]:
prefix = 'lammps_simple'
input_s3_uri = sagemaker.session.Session().upload_data(path='param/', key_prefix=prefix)
print(input_s3_uri)

s3://sagemaker-ap-northeast-1-155580384669/lammps_simple


### SageMaker Processing の実行パラメータを設定

In [21]:
# S3 から SageMaker Processing インスタンスへの転送先
PROCESSING_INPUT_DIR = '/opt/ml/processing/input/'

# SageMaker Processing インスタンス内の結果出力先
PROCESSING_OUTPUT_DIR = '/opt/ml/processing/output'

In [22]:
processor = sagemaker.processing.ScriptProcessor(
    base_job_name='LAMMPS-polyethylene',
    image_uri=MY_IMAGE_URI,
    command=['python'],
    role=sagemaker.get_execution_role(),
    instance_count=1,
    instance_type='ml.g4dn.xlarge'
)

### SageMaker Processing でシミュレーションを実行
* 成功していたら

In [23]:
processor.run(
    # Processing インスタンスで実行するコード
    # S3 の URI でも OK
    # ScriptProcessor のコマンドの第一引数になる(この場合は python run.py)
    # 配置場所はデフォルトで/opt/ml/processing/input/code/の下
    code='src/run.py',
    
    # ジョブ開始前に S3 から Processing インスタンス に転送するデータの設定
    # input_s3_uri 以下にあるファイル群が PROCESSING_INPUT_DIR に配置される
    inputs=[ProcessingInput(source=input_s3_uri,destination=PROCESSING_INPUT_DIR),],
    
    # ジョブ完了後に Processing インスタンスから S3 に転送する設定
    # PROCESSING_OUTPUT_DIR に配置したものが、s3://{default bucket}/{base_job_name}-YYYY-MM-DD-HH-MI-SS-mmm/output/result/以下に配置される
    outputs=[ProcessingOutput(output_name='result',source=PROCESSING_OUTPUT_DIR),],
    
    # コマンド実行時の第 2 引数以降に設定される
    # この場合は python /opt/ml/processing/input/code/run.py --input-dir $PROCESSING_INPUT_DIR --input-equiliv-in lmp_equiliv.in …
    # となる
    arguments=[
        '--input-dir',         PROCESSING_INPUT_DIR,
        '--input-equiliv-in',  'lmp_equiliv.in',
        '--input-equiliv-sh',  'lmp_equiliv.sh',
        '--input-lmp2data-py', 'lmp2data.py',
        '--np',                '2',
        '--gpu',               '1',
        '--output-dir',        PROCESSING_OUTPUT_DIR,
    ]
)


Job Name:  LAMMPS-polyethylene-2021-10-13-15-55-28-296
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-ap-northeast-1-155580384669/lammps_simple', 'LocalPath': '/opt/ml/processing/input/', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-ap-northeast-1-155580384669/LAMMPS-polyethylene-2021-10-13-15-55-28-296/input/code/run.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'result', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-ap-northeast-1-155580384669/LAMMPS-polyethylene-2021-10-13-15-55-28-296/output/result', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]
..........................................

## シミュレーション結果の確認
### ジョブの結果詳細確認

In [14]:
# 実行したジョブの詳細確認
print(processor.latest_job.describe())

{'ProcessingInputs': [{'InputName': 'input-1',
   'AppManaged': False,
   'S3Input': {'S3Uri': 's3://sagemaker-ap-northeast-1-155580384669/lammps_simple',
    'LocalPath': '/opt/ml/processing/input/',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated',
    'S3CompressionType': 'None'}},
  {'InputName': 'code',
   'AppManaged': False,
   'S3Input': {'S3Uri': 's3://sagemaker-ap-northeast-1-155580384669/LAMMPS-polyethylene-2021-10-13-15-22-05-065/input/code/run.py',
    'LocalPath': '/opt/ml/processing/input/code',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated',
    'S3CompressionType': 'None'}}],
 'ProcessingOutputConfig': {'Outputs': [{'OutputName': 'result',
    'S3Output': {'S3Uri': 's3://sagemaker-ap-northeast-1-155580384669/LAMMPS-polyethylene-2021-10-13-15-22-05-065/output/result',
     'LocalPath': '/opt/ml/processing/output',
     'S3UploadMode': 'EndOfJob'},
    'AppMa

### ジョブの出力結果を取得

In [18]:
job_name = processor.latest_job.describe()['ProcessingJobName']
output_s3_uri = processor.latest_job.describe()['ProcessingOutputConfig']['Outputs'][0]['S3Output']['S3Uri']
print(job_name, output_s3_uri)

LAMMPS-polyethylene-2021-10-13-15-22-05-065 s3://sagemaker-ap-northeast-1-155580384669/LAMMPS-polyethylene-2021-10-13-15-22-05-065/output/result


In [19]:
# ローカル(ノートブックインスタンス)にダウンロード
!mkdir ./{job_name}
!aws s3 sync {output_s3_uri} ./{job_name}

download: s3://sagemaker-ap-northeast-1-155580384669/LAMMPS-polyethylene-2021-10-13-15-22-05-065/output/result/lmp2data_stdout.txt to LAMMPS-polyethylene-2021-10-13-15-22-05-065/lmp2data_stdout.txt
download: s3://sagemaker-ap-northeast-1-155580384669/LAMMPS-polyethylene-2021-10-13-15-22-05-065/output/result/lmp2data_stderr.txt to LAMMPS-polyethylene-2021-10-13-15-22-05-065/lmp2data_stderr.txt
download: s3://sagemaker-ap-northeast-1-155580384669/LAMMPS-polyethylene-2021-10-13-15-22-05-065/output/result/mpirun_stdout.txt to LAMMPS-polyethylene-2021-10-13-15-22-05-065/mpirun_stdout.txt
download: s3://sagemaker-ap-northeast-1-155580384669/LAMMPS-polyethylene-2021-10-13-15-22-05-065/output/result/log.lammps to LAMMPS-polyethylene-2021-10-13-15-22-05-065/log.lammps
download: s3://sagemaker-ap-northeast-1-155580384669/LAMMPS-polyethylene-2021-10-13-15-22-05-065/output/result/log.cite to LAMMPS-polyethylene-2021-10-13-15-22-05-065/log.cite
download: s3://sagemaker-ap-northeast-1-155580384669/L